In [2]:
import pandas as pd
import numpy as np

In [10]:
df_disney = pd.read_csv('raw_data/disney_plus_titles.csv')
df_netflix = pd.read_csv('raw_data/netflix_titles.csv', sep=';')

#### Dataframe `df_netflix`

In [27]:
df_netflix.info()
df_netflix.shape
df_netflix.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            8809 non-null   object        
 1   type_id       8809 non-null   object        
 2   title         8809 non-null   object        
 3   director      8809 non-null   object        
 4   cast          8809 non-null   object        
 5   country       8809 non-null   object        
 6   date_added    8797 non-null   datetime64[ns]
 7   release_year  8809 non-null   object        
 8   rating        8809 non-null   object        
 9   duration      8809 non-null   object        
 10  listed_in     8809 non-null   object        
 11  description   8807 non-null   object        
 12  source        8809 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 894.8+ KB


,id,type_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,source
0,0,1,Dick Johnson Is Dead,Kirsten Johnson,No Cast,[United States],2021-09-25,2020,PG-13,90 min,[Documentaries],"As her father nears the end of his life, filmm...",Netflix
1,1,2,Blood & Water,No Director,"[Ama Qamata, Khosi Ngema, Gail Mabalane, Thaba...",[South Africa],2021-09-24,2021,TV-MA,2 Seasons,"[International TV Shows, TV Dramas, TV Mysteries]","After crossing paths at a party, a Cape Town t...",Netflix
2,2,2,Ganglands,Julien Leclercq,"[Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nab...",No Country,2021-09-24,2021,TV-MA,1 Season,"[Crime TV Shows, International TV Shows, TV Ac...",To protect his family from a powerful drug lor...,Netflix


- Dividimos los campos `cast`, `country` y `listed_in`.

In [12]:
df_netflix['cast'] = df_netflix['cast'].str.split(', ').tolist()
df_netflix['country'] = df_netflix['country'].str.split(', ').tolist()
df_netflix['listed_in'] = df_netflix['listed_in'].str.split(', ').tolist()

- Reemplazamos los NULL por valores default en las siguientes columnas:
- `title`,  `director`,  `cast`,  `country`,	 `release_year`,  `rating`,	`duration`,	`listed_in`,	`description`.

In [14]:
df_netflix['title'] = df_netflix['title'].fillna(value='No Title')
df_netflix['director'] = df_netflix['director'].fillna(value='No Director')
df_netflix['cast'] = df_netflix['cast'].fillna(value='No Cast')
df_netflix['country'] = df_netflix['country'].fillna(value='No Country')
df_netflix['release_year'] = df_netflix['release_year'].fillna(value=0)
df_netflix['rating'] = df_netflix['rating'].fillna(value='No Rating')
df_netflix['duration'] = df_netflix['duration'].fillna(value='No Duration')
df_netflix['listed_in'] = df_netflix['listed_in'].fillna(value='No Listed')

- `show_id` y `release_year` deberían ser de tipo entero.

In [15]:
#Creamos una nueva columna llamada 'id' y eliminamos 'show_id'
df_netflix.insert(loc=0, column='id', value=range(len(df_netflix)))
df_netflix.drop('show_id', inplace=True, axis=1)

In [16]:
#Vemos que, en release_year, existen datos nulos (NaN) y valores erróneos (40 min)
df_netflix['release_year'].unique()

array(['2020', '2021', '1993', '2018', '1996', '1998', '1997', '2010',
       '2013', '2017', '1975', '1978', '1983', '1987', '2012', '2001',
       '2014', '2002', '2003', '2004', '2011', '2008', '2009', '2007',
       '2005', '2006', '1994', '2015', '2019', '2016', '1982', '1989',
       '1990', '1991', '1999', '1986', '1992', '1984', '1980', '1961',
       '2000', '1995', '1985', '1976', '1959', '1988', '1981', '1972',
       '1964', '1945', '1954', '1979', '1958', '1956', '1963', '1970',
       '1973', '1925', '1974', '1960', '1966', '1971', '1962', '1969',
       '1977', '1967', '1968', '1965', '1946', '1942', '1955', 0,
       '40 min', '1944', '1947', '1943'], dtype=object)

In [17]:
#Cambiamos el tipo de dato de release_year
df_netflix['release_year'] = df_netflix['release_year'].replace(np.nan, '0')
df_netflix['release_year'] = df_netflix['release_year'].replace('40 min', '0')

#Ahora si podemos realizar la conversión
df_netflix = df_netflix.astype({'release_year':'int'})

In [18]:
df_netflix['release_year'].dtype

dtype('int32')

- Creamos tabla separada para la descripción de `type`.
- `type` debería tener solo numeros enteros asociados a cada tipo.

In [19]:
df_netflix.type.unique()

array(['Movie', 'TV Show', nan, 'William Wyler'], dtype=object)

In [20]:
#Existen valores nulos (NaN) y erróneos ('William Wyler'). Los reemplazamos por 'No Type'
df_netflix['type'].fillna(value='No Type', inplace=True)
df_netflix['type'] = df_netflix['type'].str.replace('William Wyler', 'No Type')

In [21]:
df_netflix.type.unique()

array(['Movie', 'TV Show', 'No Type'], dtype=object)

In [22]:
#Creamos un nuevo dataframe con las descripciones de los tipos y sus id
df_type = pd.DataFrame({
    'type': df_netflix['type'].unique()
    })
df_type['type_id'] = df_type['type'].replace({'Movie':1, 'TV Show':2, 'No Type':0})
df_type

,type,type_id
0,Movie,1
1,TV Show,2
2,No Type,0


In [23]:
#Creamos una nueva columna seteada en 0 por default
df_netflix.insert(loc=1, column='type_id', value=0 )

#Reemplazamos cada type por un valor entero asociado y eliminamos 'type'
df_netflix['type_id'] = df_netflix['type'].replace({'Movie':1, 'TV Show':2})
df_netflix.drop('type', inplace=True, axis=1)

- `date_added` debería ser de tipo datetime 'yyyy-mm-dd'

In [24]:
#Reemplazamos la coma por vacío y valores erroneos por vacío
df_netflix['date_added'] = df_netflix['date_added'].str.replace(',', '')
df_netflix['date_added'] = df_netflix['date_added'].str.replace('TV-PG', '')

#Eliminamos los espacios sobrantes
df_netflix['date_added'] = df_netflix['date_added'].str.strip()

#Reemplazamos los nulos con el valor más frecuente
most_frequent_date = df_netflix['date_added'].value_counts().index[0]
df_netflix[df_netflix['date_added'].str.strip()==''] = most_frequent_date

#Convertimos a tipo datetime
df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'], format='%B %d %Y')

In [25]:
print(df_netflix.date_added[0], df_netflix.date_added.dtypes)

2021-09-25 00:00:00 datetime64[ns]


- Por último, insertamos una columna que indica ser Netflix.

In [26]:
df_netflix['source'] = 'Netflix'

In [ ]:
################################################################

#### Dataframe `df_disney`

In [36]:
df_disney.info()
df_disney.shape
df_disney.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1450 non-null   object        
 1   type_id       1450 non-null   object        
 2   title         1450 non-null   object        
 3   director      1450 non-null   object        
 4   cast          1450 non-null   object        
 5   country       1450 non-null   object        
 6   date_added    1447 non-null   datetime64[ns]
 7   release_year  1450 non-null   object        
 8   rating        1450 non-null   object        
 9   duration      1450 non-null   object        
 10  listed_in     1450 non-null   object        
 11  description   1450 non-null   object        
 12  source        1450 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 147.4+ KB


,id,type_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,source
0,0,1,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","[Chris Diamantopoulos, Tony Anselmo, Tress Mac...",No Country,2021-11-26,2016,TV-G,23 min,"[Animation, Family]",Join Mickey and the gang as they duck the halls!,Disney
1,1,1,Ernest Saves Christmas,John Cherry,"[Jim Varney, Noelle Parker, Douglas Seale]",No Country,2021-11-26,1988,PG,91 min,[Comedy],Santa Claus passes his magic bag to a new St. ...,Disney
2,2,1,Ice Age: A Mammoth Christmas,Karen Disher,"[Raymond Albert Romano, John Leguizamo, Denis ...",[United States],2021-11-26,2011,TV-G,23 min,"[Animation, Comedy, Family]",Sid the Sloth is on Santa's naughty list.,Disney


- Dividimos los campos `cast`, `country` y `listed_in`.

In [29]:
df_disney['cast'] =      df_disney['cast'].str.split(', ').tolist()
df_disney['country'] =   df_disney['country'].str.split(', ').tolist()
df_disney['listed_in'] = df_disney['listed_in'].str.split(', ').tolist()

- Reemplazamos los NULL por valores default en las siguientes columnas:
- `title`,  `director`,  `cast`,  `country`,	 `release_year`,  `rating`,	`duration`,	`listed_in`,	`description`.

In [30]:
df_disney['title'] =        df_disney['title'].fillna(value='No Title')
df_disney['director'] =     df_disney['director'].fillna(value='No Director')
df_disney['cast'] =         df_disney['cast'].fillna(value='No Cast')
df_disney['country'] =      df_disney['country'].fillna(value='No Country')
df_disney['release_year'] = df_disney['release_year'].fillna(value=0)
df_disney['rating'] =       df_disney['rating'].fillna(value='No Rating')
df_disney['duration'] =     df_disney['duration'].fillna(value='No Duration')
df_disney['listed_in'] =    df_disney['listed_in'].fillna(value='No Listed')

- `show_id` debería ser de tipo INT

In [31]:
#Creamos una nueva columna llamada 'id' y eliminamos 'show_id'
df_disney.insert(loc=0, column='id', value=range(len(df_disney)))
df_disney.drop('show_id', inplace=True, axis=1)

- `type` debería contener solo el id correspondiente al tipo de show
- la descripción del tipo de show debería estar en una tabla separada

In [32]:
#Creamos una nueva columna seteada en 0 por default
df_disney.insert(loc=1, column='type_id', value=0 )

#Reemplazamos cada type por un valor entero asociado y eliminamos 'type'
df_disney['type_id'] = df_disney['type'].replace({'Movie':1, 'TV Show':2})
df_disney.drop('type', inplace=True, axis=1)

- `date_added` debería ser de tipo DATE y estar en formato dd/mm/yyyy

In [33]:
#Reemplazamos los nulos con el valor más frecuente
most_frequent_date = df_disney['date_added'].value_counts().index[0]
df_disney[df_disney['date_added'].str.strip()==''] = most_frequent_date

In [34]:
#Reemplazamos la coma por vacío
df_disney['date_added'] = df_disney['date_added'].str.replace(',', '')

#Convertimos a tipo datetime
df_disney['date_added'] = pd.to_datetime(df_disney['date_added'], format='%B %d %Y')

- Por último, insertamos una columna que indica ser Disney.

In [35]:
df_disney['source'] = 'Disney'